In [3]:
!git clone https://github.com/neccam/slt.git

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [4]:
%cd slt

/content/slt


In [5]:
!chmod +x data/download.sh
!data/download.sh

--2020-12-29 13:09:07--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  19.4MB/s    in 99s     

2020-12-29 13:10:46 (18.7 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2020-12-29 13:10:46--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: ‘phoenix14t.pami0.dev’

phoenix14t.pami0.de 100%[===================>] 124.77M  20.0MB/s    in 6.9s    

2020-12-29 13:10:54 (18.2 MB/s) - ‘phoenix14t.pami0.dev’ saved [1308312

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

In [7]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 112kB 13.6MB/s 
     |████████████████████████████████| 112kB 19.6MB/s 
     |████████████████████████████████| 163kB 21.6MB/s 
     |████████████████████████████████| 399kB 16.7MB/s 
     |████████████████████████████████| 2.3MB 35.8MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 2.4MB 61.4MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 296kB 61.0MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
     |████████████████████████████████| 13.1MB 61.9MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 20.2MB 153kB/s 
     |█

In [8]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

!unzip -q ../drive/MyDrive/slt/keypoints/dev_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/train_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/test_features.zip -d data

In [9]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:49<00:00, 142.20it/s]


In [10]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami2.{}".format(s)

  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [01:33<00:00, 75.52it/s]


In [11]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(3)]

list_doc['data']['feature_size'] = [1024, 417, 1024]
list_doc['model']['model_dir'] = "all_early_fusion"
list_doc['training']['recognition_loss_weight'] = 20


In [13]:
with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

In [14]:
!python -m signjoey train configs/sign.yaml

Streaming output truncated to the last 5000 lines.
	BLEU-4 9.68	(BLEU-1: 29.39,	BLEU-2: 17.75,	BLEU-3: 12.37,	BLEU-4: 9.68)
	CHRF 27.43	ROUGE 32.67
2020-12-29 13:33:03,149 Logging Recognition and Translation Outputs
2020-12-29 13:33:03,149 ========================================================================================================================
2020-12-29 13:33:03,149 Logging Sequence: dev/29October_2009_Thursday_tagesschau-1011
2020-12-29 13:33:03,149 	Gloss Reference :	JETZT WETTER WIE-AUSSEHEN MORGEN DREISSIG OKTOBER
2020-12-29 13:33:03,149 	Gloss Hypothesis:	***** ****** ************ ****** ******** *******
2020-12-29 13:33:03,149 	Gloss Alignment :	D     D      D            D      D        D      
2020-12-29 13:33:03,149 	--------------------------------------------------------------------------------------------------------------------
2020-12-29 13:33:03,151 	Text Reference  :	und nun die wettervorhersage für morgen freitag    den dreißigsten oktober .
2020-12-29 1

In [15]:
!python -m signjoey test configs/sign.yaml

2020-12-29 14:29:39.388838: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-29 14:29:46.500680: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-29 14:29:46.534006: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-29 14:29:46.601464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-29 14:29:46.602149: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 836.37GiB/s
2020-12-29 14:29:46.602190: I tensorflow/stream_executor/platform/defau

In [ ]:
!cp -r ./sign_sample_model "../drive/MyDrive/slt/models/all_fusion" 

cp: cannot stat './sign_sample_model': No such file or directory
